In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from scipy import sparse
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation'
%ls

/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation
data/      Metric.py  __pycache__/  Result.py
Loader.py  model/     README.md     test.ipynb


In [4]:
import Loader as ld
import Metric as me
import Result as rs

In [5]:
%cd '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF'
%ls

/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF
best_model/  MF.py  __pycache__/


In [6]:
import MF as mf

# 1. 학습
## 1-1. 데이터 로드

In [124]:
DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/'
loader1 = ld.Loader(DIR, 1, 4)
loader1.load_dataset()

(24032, 2661)

In [8]:
train = loader1.formatting(0)
valid_X, valid_y  = loader1.formatting(1)
test_X, test_y  = loader1.formatting(2)

664975


In [9]:
train_usr, train_prb, train_entry = shuffle(train[0], train[1], train[2])
train_usr = np.array(train_usr).reshape(-1,1)
train_prb = np.array(train_prb).reshape(-1,1)
train_entry = np.array(train_entry).reshape(-1,1)

##1-2. 모델 학습

In [12]:
mf = mf.MF(loader1.users_no, loader1.prob_no)
model = mf.get_model()

In [13]:
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-2)
model.compile(optimizer=optimizer, loss='mse')

In [22]:
train_N = len(train_usr)-1
valid_N = len(valid_X)-1
epochs = 1
batch_size = 1024
rc_vad = []
hr_vad = []
best_eval = -1
best_epoch = -1

MODEL_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF/best_model'

In [42]:
for epoch in range(epochs):
  '''for i in range(0, train_N, batch_size):
    idxlist = range(i, min(i+batch_size, train_N))
    model.fit([train_usr[idxlist], train_prb[idxlist]], train_entry[idxlist],verbose=1)'''
  
  hit_rate = 0
  recall = []
  for i in range(0, valid_N, batch_size):
    idxlist = range(i, min(i+batch_size, valid_N))
    valid_batch = valid_X.iloc[loader1.get_idx(valid_X, idxlist)]
    if len(valid_batch) <= 0:
      continue

    valid_X_usr = np.array(valid_batch['handle'].tolist()).reshape(-1,1)
    valid_X_prb = np.array(valid_batch['problemId'].tolist()).reshape(-1,1)

    X_pred = model.predict([valid_X_usr, valid_X_prb])
    valid_batch['pred'] = X_pred

    valid_y_batch = valid_y[0].iloc[loader1.get_idx(valid_y[0], valid_batch['handle'].tolist())]

    pred = sparse.csr_matrix((valid_batch['pred'], \
                             (valid_batch['handle'], valid_batch['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()
    heldout = sparse.csr_matrix((np.ones_like(valid_y_batch['handle']), \
                             (valid_y_batch['handle'], valid_y_batch['problemId'])),\
                             dtype='float64', shape=(loader1.users_no, loader1.prob_no)).toarray()

    recall.append(me.recall_at_k(pred, heldout, k=20))
    hit_rate += me.hit_rate_at_k(pred, heldout, k=20)

  recall_ = np.concatenate(recall)
  recall_[np.isnan(recall_)]=0 
  recall_ = recall_.mean()
  rc_vad.append(recall_)
  print(epoch, recall_)

  hit_rate_ = hit_rate/loader1.users_no
  hr_vad.append(hit_rate_)
  print(epoch, hit_rate_)

  if hit_rate_ > best_eval:
    model.save(MODEL_DIR)
    best_epoch = epoch
    best_eval = hit_rate_
    print(epoch, best_eval)

[[   3]
 [   3]
 [   3]
 ...
 [1008]
 [1008]
 [1008]]


ValueError: ignored

# 2. 예측

In [9]:
MODEL_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/Recommendation/model/MF/best_model'
DATA_DIR = '/content/drive/MyDrive/(22-1)캡스톤/recomm/data/preprocessed/data1'
result = rs.Result(MODEL_DIR, DATA_DIR, "1", 1024)
print(result.get_result(0))

[17207 13908  1895  3023  3034 23969  5026  5087  8370  9289]
